> 决策树是一种预测模型；决策树仅有单一输出，如果有多个输出，可以分别建立独立的决策树以处理不同的输出。如图所示：


![](http://img.blog.csdn.net/20161121193225546)

## 基本概念

### 1.信息熵
信息熵(`information entropy`)定义为离散随机事件出现的概率，一个系统**越是稳定**，信息熵就**越低**，反之一个系统**越是混乱**，则它的信息熵**越高**；所以信息熵可以被认为是系统稳定性的一个度量。

*如何计算信息熵？*

对于有`K`个 类别的分类问题来说，假定样本集合`D`中第`k`类样本所占的比例为`p_k(k=1,2,...K)`， 则样本集合`D`的信息熵为:
![image.png](./docs/xinxishang.png)

### 2.信息增益
信息增益是针对一个特定的分支标准，在该分支下计算原有数据的信息熵与引入该分支标准后的信息熵之差；

**定义**

![image.png](./docs/xinxizengyi.png)

例如：
![](./docs/xinxizengyi_lizi.png)

通过表1中天气的4个特征(`outlook, temperature, humidity, windy`)来分类目标(`play, not play`)

![image.png](./docs/xixinzengyi_jie.png)

## 决策树构造

### ID3算法
> ID3算法是一种贪心算法，用来构造决策树。ID3算法中根据特征选择和信息增益评估分支标准，每次选择**信息增益最大**的特征作为分支标准。

#### 算法思想
ID3算法的基本思想是：首先计算出原始数据集的信息熵，然后依次将数据中的每一个特征作为分支标准，并计算其相对于原始数据的信息增益，选择最大信息增益的分支标准来划分数据，因为信息增益越大，区分样本的能力就越强，越具有代表性。重复上述过程从而生成一棵决策树，很显然这是一种自顶向下的贪心策略。
#### 算法缺陷
+ 使用信息增益有一个缺点，那就是它会偏向于具有大量值的属性，也就是说在训练集中某个属性所取得不同值的个数越多，则越有可能选择它来作为分类属性，而这么做有时候是没有意义的；
+ ID3算法不能处理连续分布的数据特征；

### C4.5算法
> C4.5算法是对ID3算法的改进，它克服了ID3的**两个**缺点；对于离散的特征，C4.5算法**不直接使用**信息增益，而是使用**增益率(`gain ratio`)**来选择最优的分支标准；

#### 增益率
定义：

![image.png](./docs/zengyilv.png?raw=true)

#### 连续值属性处理方法
对于连续属性值，C4.5算法处理的方法是**先把连续属性值转换为离散属性在进行处理；**虽然本质上属性的取值是连续的，但对于有限的采样数据它是离散的，如果有N条样本，那么我们有N-1种离散化的方法：<=vj的分到左子树，>vj的分到右子树。计算这N-1种情况下最大的信息增益率。在离散属性上只需要计算1次信息增益率，而在连续属性上却需要计算N-1次，计算量是相当大的。通过以下办法可以减少计算量：对于连续属性先按大小进行排序，只有在分类发生改变的地方才需要切开。例如：
![image.png](./docs/C4.5.png?raw=true)

对表中的连续属性Temperature进行分析，首先对样本根据Temperature属性值进行排序，结果如上表所示。在分类改变的地方(表中红线部分)进行切开，本来有13种离散化的情况，现在只需计算7种。

如果利用增益率来选择连续值属性的分界点，会导致一些副作用。分界点将样本分成两个部分，这两个部分的样本个数之比也会影响增益率。根据增益率公式，我们可以发现，当分界点能够把样本分成数量相等的两个子集时（我们称此时的分界点为等分分界点），增益率的抑制会被最大化，因此等分分界点被过分抑制了。子集样本个数能够影响分界点，显然不合理。因此在决定分界点是还是采用增益这个指标，而选择属性的时候才使用增益率这个指标。

### CART算法
> 上面介绍的ID3，C4.5算法都是基于信息熵来进行选取划分节点的，主要用于**分类问题**，而CART决策树全称为分类回归树(Classification And Regression Tree)，分类回归问题都可以使用

CART和前面两种算法不同的地方是，在每一次节点做判断时，只考虑二分类的情况，即使特征能够取到多个值（比如属性颜色有红、黄、蓝三种取值，ID3和C4.5直接就划分为红、黄、蓝三个子类，而CART只能在一次划分时划分为是不是红（黄、蓝）然后再进行判断。

#### 基尼指数:$$ Gini(D)=1- \sum_{k=1}^N p_{k}^2$$
其中$p_{k}$和前文相同，表示第k类样本所占的比例，基尼指数同样可以视为衡量选取划分特征有效程度的一个指标。当样本非常不均匀的时候，基尼指数较小，当样本均匀的时候，基尼指数较大。**因此，我们的目标是：选取划分后基尼指数最小的属性（最能够区分样本的属性）进行划分。**

#### 定义属性a的基尼指数:$$ Gini_{a} = \sum_{v=1}^V \frac{|D^v|}{|D|}Gini(D^v)$$
因此,我们选择$Gini_{a}$ 最小的特征进行划分

[例子](https://zhuanlan.zhihu.com/p/32003259)

In [6]:
# _*_ coding:utf8 _*_

"""
C4.5构建DTree
"""
from math import log
import operator
import matplotlib.pyplot as plt

# 创建数据
def createDataSet():
    dataSet = [[0, 0, 0, 0, 'N'],
               [0, 0, 0, 1, 'N'],
               [1, 0, 0, 0, 'Y'],
               [2, 1, 0, 0, 'Y'],
               [2, 2, 1, 0, 'Y'],
               [2, 2, 1, 1, 'N'],
               [1, 2, 1, 1, 'Y']]
    labels = ['outlook', 'temperature', 'humidity', 'windy']
    return dataSet, labels

# 计算熵
def calcEnt(dataset):
    numEntries = len(dataset)
    labelCounts = {}
    for featvec in dataset:
        currentLabel = featvec[-1]
        if currentLabel not in labelCounts:
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1

    ent = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key])/numEntries
        ent -= prob * log(prob,2)
    return ent

# 采用多数判决的方法决定子节点的分类
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount:
            classCount[vote]=0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reversed=True)
    return sortedClassCount[0][0]

# 选择最大的gain ratio对应的feature
def chooseBestFeature(dataset):
    numFeatures = len(dataset[0]) - 1
    baseEnt = calcEnt(dataset)        # 整个dataset的熵
    bestInfoGainRatio = 0.0
    bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataset]
        uniqueVals = set(featList)
        newEnt = 0.0
        splitInfo = 0.0
        for value in uniqueVals:
            subDataset = splitDataSet(dataset, i, value)  #每个唯一值对应的剩余feature的组成子集
            prob = len(subDataset)/float(len(dataset))
            newEnt += prob * calcEnt(subDataset)
            splitInfo += -prob * log(prob, 2)
        infoGain = baseEnt - newEnt
        if (splitInfo == 0):
            continue
        infoGainRatio = infoGain/splitInfo
        if (infoGainRatio > bestInfoGainRatio):
            bestInfoGainRatio = infoGainRatio
            bestFeature = i
    return bestFeature

# 划分数据，为下一层计算准备
def splitDataSet(dataset, axis, value):
    retDataset = list()
    for featvec in dataset:
        if featvec[axis] == value:
            reduceFeatVec = featvec[:axis]
            reduceFeatVec.extend(featvec[axis+1:])
            retDataset.append(reduceFeatVec)

    return retDataset

# 构建DTree
def createTree(dataset, labels):
    classList = [example[-1] for example in dataset]
    if classList.count(classList[0]) == len(classList):
        # classList所有元素都相等，即类别完全相同，停止划分
        return classList[0]
    if len(dataset[0]) == 1:
        return majorityCnt(classList)
    bestFeature = chooseBestFeature(dataset)
    # 选择最大的gain ratio对应的feature
    bestFeatLabel = labels[bestFeature]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeature])

    featValues = [example[bestFeature] for example in dataset]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataset, bestFeature, value), subLabels)

    return myTree

dataSet, labels = createDataSet()
labels_tmp = labels[:]
desicionTree = createTree(dataSet, labels_tmp)
print(desicionTree)
#treePlotter.createPlot(desicionTree)

{'outlook': {0: 'N', 1: 'Y', 2: {'windy': {0: 'Y', 1: 'N'}}}}
